In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Data transforms ---
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# --- Datasets ---
train_dataset = datasets.ImageFolder(
    r"C:\Users\HP\OneDrive\Desktop\projectWorks\6MoodImage\moodcnnProgram\model\train",
    transform=train_transforms
)
test_dataset = datasets.ImageFolder(
    r"C:\Users\HP\OneDrive\Desktop\projectWorks\6MoodImage\moodcnnProgram\model\test",
    transform=test_transforms
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

num_classes = len(train_dataset.classes)
print("Classes:", train_dataset.classes)

# --- Pretrained ResNet18 ---
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False  # Freeze base layers

# Replace the final FC layer for your 3 moods
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# --- Loss and Optimizer ---
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# --- Train ---
epochs = 10
train_losses, train_accs = [], []

for epoch in range(epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()
    
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct / total
    train_losses.append(epoch_loss)
    train_accs.append(epoch_acc)
    
    print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} - Acc: {epoch_acc:.4f}")

# --- Save model ---
torch.save({
    'model_state_dict': model.state_dict(),
    'class_to_idx': train_dataset.class_to_idx
}, r"C:\Users\HP\OneDrive\Desktop\projectWorks\6MoodImage\moodcnnProgram\emosion_backend\api\mood_cnn_resnet18.pth")

# --- Plot ---
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(train_losses, label='Loss', color='red')
plt.title("Training Loss")
plt.subplot(1,2,2)
plt.plot(train_accs, label='Accuracy', color='blue')
plt.title("Training Accuracy")
plt.show()


Classes: ['happy', 'neutral', 'sad']
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\HP/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


c:\Users\HP\anaconda3\envs\moodcnn\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HP\anaconda3\envs\moodcnn\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100.0%


Epoch 1/10 - Loss: 0.9941 - Acc: 0.5094
Epoch 2/10 - Loss: 0.9263 - Acc: 0.5634
Epoch 3/10 - Loss: 0.9138 - Acc: 0.5702


KeyboardInterrupt: 

In [ ]:
svpath=r"C:\Users\HP\OneDrive\Desktop\projectWorks\6MoodImage\moodcnnProgram\emosion_backend\api\mood_cnn_resnet18.pth"
checkpoint = torch.load(svpath, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
class_to_idx = checkpoint['class_to_idx']
idx_to_class = {v: k for k, v in class_to_idx.items()}  # reverse mapping

def predict_mood(image_path):
    img = Image.open(image_path).convert("L")
    img = test_transforms(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(img)
        pred = torch.argmax(output, dim=1).item()
    return idx_to_class[pred]


In [ ]:
mood = predict_mood("C:/Users/HP/OneDrive/Desktop/projectWorks/6MoodImage/moodcnnProgram/model/upload_photos/emma.jpg")
print("Predicted mood:", mood)